In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.2/410.2 MB 3.2 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import tensorflow as tf
import spacy
import os
import json
import codecs
import pickle
import random
import csv

2022-09-19 02:06:34.905731: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-19 02:06:35.045846: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-19 02:06:35.045887: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-19 02:06:35.082138: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-19 02:06:37.362120: W tensorflow/stream_executor/pla

construyendo el dataset

In [3]:
nlp = spacy.load('es_dep_news_trf')


data = json.loads(codecs.open(
    'categorias.json', 'r', encoding='utf-8').read())
print(data)


{'categorias': [{'categoria': 'saludos', 'sentencias': ['hola', 'hey', 'buenos dias', 'hola, que tal'], 'respuestas': ['hola', 'buenos días', 'hola, en que puedo ayudarte?']}, {'categoria': 'despedida', 'sentencias': ['hasta pronto', 'tenga un buen dia', 'nos vemos', 'hasta luego', 'adios', 'fue un gusto hablar contigo'], 'respuestas': ['hasta luego', 'tenga un buen dia usted', 'nos vemos, me encanto conversar contigo']}, {'categoria': 'nombre', 'sentencias': ['cual es tu nombre', 'como te llamas', 'quien eres', 'como te conocen'], 'respuestas': ['mi nombre es Neural', 'me llamo Neural', 'soy Neural una ia amistosa', 'todos me conocen como Neural tu ia amigable']}, {'categoria': 'edad', 'sentencias': ['cuantos años tienes', 'cual es tu edad', 'que edad tienes', 'cuan viejo eres', 'hace cuanto estas creado', 'desde cuando te crearon', 'cuando naciste'], 'respuestas': ['tengo 6 meses de vida', '6 meses es lo que tengo de creada', 'Angel me creo hace 6 meses']}, {'categoria': 'funcionamie

In [4]:
palabras = set()
categorias = []
categorias_lemmas = []
ignorar = ["?", "!", ",", ".", ":"]
for categoria in data["categorias"]:
    lematizacion = set()
    for sentencia in categoria["sentencias"]:
        print("Procesando->", sentencia)
        tokens = nlp(sentencia)
        lemmas = [tok.lemma_.lower()
                  for tok in tokens if tok.text not in ignorar]
        lematizacion = lematizacion.union(set(lemmas))
        categorias_lemmas.append([lematizacion, categoria["categoria"]])
    palabras = palabras.union(lematizacion)
    categorias.append(categoria["categoria"])
print(palabras)
print(categorias_lemmas)

Procesando-> hola
Procesando-> hey
/shared-libs/python3.9/py/lib/python3.9/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Procesando-> buenos dias
Procesando-> hola, que tal
Procesando-> hasta pronto
Procesando-> tenga un buen dia
Procesando-> nos vemos
Procesando-> hasta luego
Procesando-> adios
Procesando-> fue un gusto hablar contigo
Procesando-> cual es tu nombre
Procesando-> como te llamas
Procesando-> quien eres
Procesando-> como te conocen
Procesando-> cuantos años tienes
Procesando-> cual es tu edad
Procesando-> que edad tienes
Procesando-> cuan viejo eres
Procesando-> hace cuanto estas creado
Procesando-> desde cuando te crearon
Procesando-> cuando naciste
Procesando-> a que te dedicas
Procesando-> que haces
Procesando-> como trabajas
Procesando-> como funcionas
{'tu', 'gusto', 'tal', 'cuan', 'dedic

In [5]:
palabras = sorted(palabras)
categorias = sorted(categorias)
pickle.dump(palabras, open('palabras.pkl', 'wb'))
pickle.dump(categorias, open('categorias.pkl', 'wb'))
print(palabras)
print(categorias)

['a', 'adios', 'año', 'buen', 'como', 'conocer', 'creado', 'crear', 'cual', 'cuan', 'cuando', 'cuanto', 'dedicar', 'desde', 'dia', 'edad', 'estar', 'funcionar', 'gusto', 'hablar', 'hacer', 'hasta', 'hey', 'hola', 'llamar', 'luego', 'nacer', 'nombre', 'pronto', 'que', 'quien', 'ser', 'tal', 'tener', 'tenga', 'trabajar', 'tu', 'tú', 'uno', 'ver', 'viejo', 'yo']
['despedida', 'edad', 'funcionamiento', 'nombre', 'saludos']


In [6]:
training = []
for arr in categorias_lemmas:
    lemmas = arr[0]
    categoria = arr[1]
    salida = [0] * len(categorias)
    salida[categorias.index(categoria)] = 1
    index_categoria = categorias.index(categoria)
    row = []
    for word in palabras:
        if word in lemmas:
            row.append(1)
        else:
            row.append(0)
    training.append([row, salida])
random.shuffle(training)
training = np.array(training, dtype=object)
train_x = list(training[:,0])
train_y = list(training[:,1])
print(train_x)
print(train_y)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1], [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 

In [7]:
print(len(palabras))
print(len(categorias))


42
5


In [8]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(len(palabras),)))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(.5))
model.add(tf.keras.layers.Dense(len(categorias), activation='softmax'))
opt = tf.keras.optimizers.SGD(learning_rate=0.04, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=opt, metrics=['accuracy'])
hist = model.fit(np.array(train_x),np.array(train_y), batch_size=4000, epochs=200, verbose=1)
model.save('chatbot_model.h5', hist)
print("Done")

Epoch 1/200
2022-09-19 02:06:52.186842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
1/1 [==============================] - 0s 461ms/step - loss: 1.7001 - accuracy: 0.0800
Epoch 2/200
1/1 [==============================] - 0s 4ms/step - loss: 1.6267 - accuracy: 0.1600
Epoch 3/200
1/1 [==============================] - 0s 3ms/step - loss: 1.5105 - accuracy: 0.2800
Epoch 4/200
1/1 [==============================] - 0s 3ms/step - loss: 1.5227 - accuracy: 0.4000
Epoch 5/200
1/1 [==============================] - 0s 3ms/step - loss: 1.3167 - accuracy: 0.6000
Epoch 6/200
1/1 [==============================] - 0s 3ms/step - loss: 1.2930 - accuracy: 0.4800
Epoch 7/200
1/1 [==============================] - 0s 42ms/step - 

In [9]:
import random
import json
import pickle
import numpy as np
import spacy
import codecs
import tensorflow as tf

nlp = spacy.load('es_dep_news_trf')

intentos = json.loads(codecs.open(
    'categorias.json', 'r', encoding='utf-8').read())

palabras = pickle.load(open('palabras.pkl', 'rb'))
categorias = pickle.load(open('categorias.pkl', 'rb'))
model: tf.keras.models.Model = tf.keras.models.load_model('chatbot_model.h5')
ignorar = ["?", "!", ",", ".", ":"]


def entradas(sentencia):
    tokens = nlp(sentencia)
    lemmas = set([tok.lemma_.lower()
                  for tok in tokens if tok.text not in ignorar])
    entrada_palabras = [0] * len(palabras)
    for word in lemmas:
        for i, palabra in enumerate(palabras):
            if word == palabra:
                entrada_palabras[i] = 1
    return np.array(entrada_palabras)


def prediccion(sentencia):
    req = entradas(sentencia)
    res = model.predict(np.array([req]))[0]
    error = 0.25
    resultados = [[i, r] for i, r in enumerate(res) if r > error]
    resultados.sort(key=lambda x: x[1], reverse=True)
    listado = []
    for r in resultados:
        listado.append(
            {'categoria': categorias[r[0]], 'probabilidad': str(r[1])})
    print("prediccion-listado: ", listado)
    return listado


def respuesta(sentencia):

    categoria = prediccion(sentencia)[0]["categoria"]
    datos = intentos["categorias"]
    res = "No tengo muchos temas para hablar, hablemos de otra cosa"
    for i in datos:
        if i["categoria"] == categoria:
            res = random.choice(i["respuestas"])
            break
    return res


print("ChatBot Listo")
salir = True
print("Chatbot, para cerrar el chat escriba 'salir'")
while salir:
    
    message = input("")
    print(message)
    if message == "salir":
        salir = False
        print("Adios!!.")
    else:
        print(respuesta(message))

ChatBot Listo
Chatbot, para cerrar el chat escriba 'salir'
hola
1/1 [==============================] - 0s 126ms/step
prediccion-listado:  [{'categoria': 'saludos', 'probabilidad': '0.9984871'}]
hola, en que puedo ayudarte?
cuantos años tienes
1/1 [==============================] - 0s 69ms/step
prediccion-listado:  [{'categoria': 'edad', 'probabilidad': '0.9999684'}]
6 meses es lo que tengo de creada
que puedes 
1/1 [==============================] - 0s 70ms/step
prediccion-listado:  [{'categoria': 'funcionamiento', 'probabilidad': '0.7002065'}, {'categoria': 'saludos', 'probabilidad': '0.2504473'}]
soy una ia amigable para chatear con personas
hasta luego
1/1 [==============================] - 0s 66ms/step
prediccion-listado:  [{'categoria': 'despedida', 'probabilidad': '0.9933316'}]
tenga un buen dia usted
s
1/1 [==============================] - 0s 16ms/step
prediccion-listado:  [{'categoria': 'saludos', 'probabilidad': '0.67279637'}]
hola, en que puedo ayudarte?
salir
Adios!!.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=829e846e-3785-491b-bcb1-f78f25dd8f99' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>